In [2]:
%pip install sqlite-vec

  Using cached sqlite_vec-0.1.6-py3-none-win_amd64.whl.metadata (198 bytes)
Using cached sqlite_vec-0.1.6-py3-none-win_amd64.whl (281 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sqlite3
import sqlite_vec

db = sqlite3.connect("../../data/cleaned_with_bge_m3.db")
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)

In [4]:
db.execute("SELECT vec_version()").fetchone()

('v0.1.6',)

In [27]:
db.execute("DROP TABLE IF EXISTS publications_vec_all_MiniLM_L6_v2")
db.execute("""
CREATE VIRTUAL TABLE publications_vec_all_MiniLM_L6_v2 USING vec0(
    publication_id INTEGER PRIMARY KEY,
    embedding FLOAT[384] distance_metric=cosine
)
""")
db.commit()

In [28]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

print(model)
print(model.get_sentence_embedding_dimension())

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
384


In [ ]:
# Query to get all tables in the database
tables = db.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()

# Print the table names
print("Tables in the database:")
for table in tables:
    print(f"- {table[0]}")

# Also get the virtual tables
virtual_tables = db.execute("SELECT name FROM sqlite_master WHERE type='table' AND sql LIKE '%VIRTUAL TABLE%'").fetchall()
if virtual_tables:
    print("\nVirtual tables:")
    for table in virtual_tables:
        print(f"- {table[0]}")


Tables in the database:
- users
- publications
- sqlite_sequence
- publication_user_mapping
- publications_vec_bge_m3_info
- publications_vec_bge_m3_chunks
- publications_vec_bge_m3_rowids
- publications_vec_bge_m3_vector_chunks00
- publications_vec_all_MiniLM_L6_v2_info
- publications_vec_all_MiniLM_L6_v2_chunks
- publications_vec_all_MiniLM_L6_v2_rowids
- publications_vec_all_MiniLM_L6_v2_vector_chunks00
- publications_vec_bge_m3
- publications_vec_all_MiniLM_L6_v2

Virtual tables:
- publications_vec_bge_m3
- publications_vec_all_MiniLM_L6_v2


In [31]:
tables = db.execute("SELECT * FROM publications_vec_all_MiniLM_L6_v2 LIMIT 1").fetchall()

print(tables)

[(1, b'\x9c\xa5\x9d<\xc3\xba&\xba \xca\xd4<\x9a\xb2V\xbd\x07\x9d\x91\xbc\xb1\x9fG=\xa1oE<K_\x10\xbd\xf3\x941=j\x02f\xbd\xb8z\x88<&\x16\xa8\xbd\xf46\xde\xbc=\xa3\x86\xbdA\xf8\xd5\xbc%\xf6@\xbdz\x19<=\xda\xc9\xca<\x95\xb1Q\xbcA8\t\xbdg\xea\xe3=Y\x8bi;\x89hj=\xaf\xc2K=\xc3\x8a\xd1<\x04)\x1d<]\x9d\x8d\xbc@#\x1c=\x1eR\xd0<\xa3r%\xbc\xe7\x99\xac\xbd\x9f\xf7\xa0=\x85\xdf^=8x\xfd<\x1c\xfe\xbd\xbc,\xce\xd4=\x00\x83\n\xbd\x93\xdf\x16;c5\x1e\xbb\xb0\x123=\xd6Tx\xbd\xaf\x001\xbd\nY\x14\xbd4\xb1X\xbc}C[=v\x18z\xbc!m\xd2\xbd\xc2\xd6\x0f<\xe8\x8e\r\xbd\xb9\x05#<\xf3\x81\xf5\xbd\xd5\xe9\x00=\xbcfr\xbd3\xd3\x9d\xbcQ\xda\xae<\'\xbat\xbd\xb9"J=k\x92 =\xe8\xd1\x19=\xfa\xb9\x89\xbc?\x9e\xad\xbb\xd0O\xbf\xbcU\xf2\x0b<M/\xce=\x19\xbb\xd3=w\xa9\xc4\xbdMaR<\x0c\x12.;x5\x9d\xbd\x16~s<1\xf7\xa9\xbd\xca\xfe\xcd<g?\x1f\xbbB\xfd\x83=\xcb\x82\x08\xbe\xba\xf1\xd2\xbc\x994\x9a<\x16)\x95\xbd\xa1M\x85\xbcl,\r\xbdh\x07\xb7=hL \xbd\xcc\xb7e=\xc2\xca\x97<f\x1a+\xbd\x03sj\xbdo6d=y@~\xbdeI3\xbdXPv=\\\xfam\xbd\xd3b\xa0\xbd\xc

In [23]:
# print(model.encode(["Hello world!"])[:10])
# print("haha")
# print(model.encode("Hello world!")[:10])

print(model.encode(["Hello world!"]).shape)
print("haha")
print(model.encode("Hello world!").shape)
# print(model.encode("Hello world!")[0])

(1, 384)
haha
(384,)


In [37]:
publications = db.execute("SELECT id, title, abstract FROM publications LIMIT 1").fetchall()
for pub_id, title, abstract in publications:
    text = f"{title}: {abstract}"
    print(text)
    print(model.encode(text).shape)
    

The Indonesian subtitles of the English utterances spoken by the characters in American Beauty: This thesis is particularly about the translation done on the subtitles of the movie entitled American Beauty. In doing the research the writer would like to find out (1) what kinds of mistakes appear in the subtitles that do not convey the meaning of the original utterances, (2) which mistake mostly occurs in the subtitles (3) what the quality of the subtitles is. To do the research, the writer would like to make use of the qualitative approach. The data of the transcription of utterances are obtained from the internet and then compared to the VCD (Video Compact Disc) entitled American Beauty. The seventy-six scenes in the film have three parts - the beginning, the conflicts, and the ending. The subtitles being analyzed are from twelve scenes which belong to the three parts, having more utterances than the others. The analysis is based on the theories of translation. The translation theorie

In [29]:
# Get publications data and generate embeddings
publications = db.execute("SELECT id, title, abstract FROM publications").fetchall()

for pub_id, title, abstract in publications:
    # Print progress every 1% processed
    if (pub_id - 1) % max(1, len(publications) // 100) == 0:
        print(f"Processing publication {pub_id} ({((pub_id - 1) / len(publications) * 100):.1f}%)")
        
    # Check if embedding already exists
    existing = db.execute(
        "SELECT 1 FROM publications_vec_all_MiniLM_L6_v2 WHERE publication_id = ?", 
        (pub_id,)
    ).fetchone()

    if existing:
        continue
    
    # Create merged string
    text = f"{title}: {abstract}"
    
    # Generate embedding
    embedding = model.encode(text)
    
    # Insert into vector table
    db.execute(
        "INSERT INTO publications_vec_all_MiniLM_L6_v2 (publication_id, embedding) VALUES (?, ?)",
        (pub_id, embedding)
    )
    
    db.commit()

db.commit()

# vacuum
db.execute("VACUUM")
print(f"Inserted {len(publications)} embeddings into publications_vec_all_MiniLM_L6_v2")

Processing publication 1 (0.0%)
Processing publication 455 (1.0%)
Processing publication 909 (2.0%)
Processing publication 1363 (3.0%)
Processing publication 1817 (4.0%)
Processing publication 2271 (5.0%)
Processing publication 2725 (6.0%)
Processing publication 3179 (7.0%)
Processing publication 3633 (8.0%)
Processing publication 4087 (9.0%)
Processing publication 4541 (10.0%)
Processing publication 4995 (11.0%)
Processing publication 5449 (12.0%)
Processing publication 5903 (13.0%)
Processing publication 6357 (14.0%)
Processing publication 6811 (15.0%)
Processing publication 7265 (16.0%)
Processing publication 7719 (17.0%)
Processing publication 8173 (18.0%)
Processing publication 8627 (19.0%)
Processing publication 9081 (20.0%)
Processing publication 9535 (21.0%)
Processing publication 9989 (22.0%)
Processing publication 10443 (22.9%)
Processing publication 10897 (23.9%)
Processing publication 11351 (24.9%)
Processing publication 11805 (25.9%)
Processing publication 12259 (26.9%)
Pr